# 物件レコメンドのスコア計算

### 方針
<li>顧客が指定したベンチマーク店舗(自社好調店舗、ライバル店舗など)との類似度から物件のスコアを計算
<li>スコア計算には４つの特徴量を使用（Geodemo,小売業売上,あさひる統計,駅距離）
<li>スコア計算には物件の絶対評価ではなく、他の空き物件との比較も含めた相対評価を行なっている

In [12]:
import warnings
import datetime
import json
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.cloud import bigquery
from scipy.spatial import distance
from tqdm import tqdm

warnings.simplefilter('ignore')

In [13]:
# BigQuery上にアップロード
project_id = 'geom-prj-property-recommend'
client = bigquery.Client()

def upload_bq(dataset,table_name,df):
    table_id = f'geom-prj-property-recommend.{dataset}.{table_name}'
    df.to_gbq(table_id)


# ベンチマーク店舗と空き物件のgeodemo、あさひる、商業集積度、駅距離を集計
def agg_geodemo_asahiru_income_station(brand_name,table_name):
    query=f'''
        with geodemo_asahiru as (
        select 
        distinct
          bld_name,pref_name,address,longitude,latitude,
        
          sum(w0_3) as w0_3,sum(w3_6) as w3_6,sum(w6_9) as w6_9,sum(w9_12) as w9_12,sum(w12_15) as w12_15,sum(w15_18) as w15_18,sum(w18_21) as w18_21,sum(w21_24) as w21_24,
          sum(h0_3) as h0_3,sum(h3_6) as h3_6,sum(h6_9) as h6_9,sum(h9_12) as h9_12,sum(h12_15) as h12_15,sum(h15_18) as h15_18,sum(h18_21) as h18_21,sum(h21_24) as h21_24,
          
          sum(totalpop) as totalpop,
          
          round(count(geodemo10=1 or null)/count(geodemo10),3) AS geodemo1,
          round(count(geodemo10=2 or null)/count(geodemo10),3) AS geodemo2,
          round(count(geodemo10=3 or null)/count(geodemo10),3) AS geodemo3,
          round(count(geodemo10=4 or null)/count(geodemo10),3) AS geodemo4,
          round(count(geodemo10=5 or null)/count(geodemo10),3) AS geodemo5,
          round(count(geodemo10=6 or null)/count(geodemo10),3) AS geodemo6,
          round(count(geodemo10=7 or null)/count(geodemo10),3) AS geodemo7,
          round(count(geodemo10=8 or null)/count(geodemo10),3) AS geodemo8,
          round(count(geodemo10=9 or null)/count(geodemo10),3) AS geodemo9,
          round(count(geodemo10=10 or null)/count(geodemo10),3) AS geodemo10,
          
        from `geomarketing-common.public_asia.asahiru_chomoku_union` asahiru
        cross join `geom-prj-property-recommend.{brand_name}.{table_name}` store
        where st_distance(bq_centroidgeom,st_geogpoint(store.longitude,store.latitude)) <=1000
        group by     
        bld_name,pref_name,address,longitude,latitude
        ),
        geodemo_asahiru_income as(
          select
          bld_name,pref_name,address,longitude,latitude,
        
          w0_3,w3_6,w6_9,w9_12,w12_15,w15_18,w18_21,w21_24,
          h0_3,h3_6,h6_9,h9_12,h12_15,h15_18,h18_21,h21_24,
          
          sum(ga.totalpop) as asahiru_totalpop,
          
          geodemo1,geodemo2,geodemo3,geodemo4,geodemo5,geodemo6,geodemo7,geodemo8,geodemo9,geodemo10,
          
          -- 収入
          sum(I_amounts) as I_amounts,
        
        from geodemo_asahiru ga
        cross join `geomarketing-common.public_asia.amounts_500mesh`
        where st_distance(geom_point,st_geogpoint(ga.longitude,ga.latitude)) <=1000
        group by   
          w0_3,w3_6,w6_9,w9_12,w12_15,w15_18,w18_21,w21_24,
          h0_3,h3_6,h6_9,h9_12,h12_15,h15_18,h18_21,h21_24,
          geodemo1,geodemo2,geodemo3,geodemo4,geodemo5,geodemo6,geodemo7,geodemo8,geodemo9,geodemo10,
          bld_name,pref_name,address,longitude,latitude
        ),
        ct as (
          select 
          bld_name,pref_name,address,longitude,latitude,
        
          geodemo1,geodemo2,geodemo3,geodemo4,geodemo5,geodemo6,geodemo7,geodemo8,geodemo9,geodemo10,
        
          I_amounts,
        
          w0_3,w3_6,w6_9,w9_12,w12_15,w15_18,w18_21,w21_24,
        
          h0_3,h3_6,h6_9,h9_12,h12_15,h15_18,h18_21,h21_24,
          asahiru_totalpop,
        from geodemo_asahiru_income
        group by 
          bld_name,pref_name,address,longitude,latitude,
          geodemo1,geodemo2,geodemo3,geodemo4,geodemo5,geodemo6,geodemo7,geodemo8,geodemo9,geodemo10,
        
          I_amounts,
        
          w0_3,w3_6,w6_9,w9_12,w12_15,w15_18,w18_21,w21_24,
        
          h0_3,h3_6,h6_9,h9_12,h12_15,h15_18,h18_21,h21_24,
          asahiru_totalpop
        ),
        station_distance as (
        select
          bld_name,pref_name,address,longitude,latitude,
        
          geodemo1,geodemo2,geodemo3,geodemo4,geodemo5,geodemo6,geodemo7,geodemo8,geodemo9,geodemo10,
        
          I_amounts,
        
          w0_3,w3_6,w6_9,w9_12,w12_15,w15_18,w18_21,w21_24,
        
          h0_3,h3_6,h6_9,h9_12,h12_15,h15_18,h18_21,h21_24,
          asahiru_totalpop,
        
          st_distance(centroidgeom,st_geogpoint(income.longitude,income.latitude)) as distance_from_station  
        
        from
          ct as income
        cross join (
          select station_name, passenger_count, centroidgeom
          from `geomarketing-common.point_of_interest.station2019` 
          where passenger_count >= 100
        ) as station
        where st_distance(st_geogpoint(longitude, latitude), centroidgeom) <= 10000
        ),
        make_rank as (
          select 
            station_distance.*,
            row_number() over(partition by bld_name order by distance_from_station) as num
          from station_distance
        )
        , station_no1 as (
        select * except(num)
        from make_rank
        where num=1 
        )
        select a.*
        from station_no1 as a
    '''
    
    df=client.query(query).to_dataframe()
    
    return df


#　人口比率
def calc_ratio_population(df,pop_target):
    df.loc[:,"asahiru_totalpop"]=df.iloc[:, 17:33].mean(axis=1)
    df.loc[:,"asahiru_totalpop_ratio"]=round((df["asahiru_totalpop"]+1)/pop_target,3)
    
    func=lambda x: x if x <= 1 else (round((-5/16*x+5/4),3) if x<=4 else 0) 
    df.loc[:,"asahiru_totalpop_ratio"]=df["asahiru_totalpop_ratio"].map(func)

    return df


# 類似度計算の関数
def cos_sim(v1, v2):

    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


def spearman(v1, v2):
    N = len(v1)

    return 1 - (6*sum((v1 -v2)**2) / (N*(N**2 - 1)))


def euclidean_distance(v1, v2):   
    return np.sqrt(np.sum((v1 - v2) ** 2))


# 事前に相対評価用のデータと比較、ランクづけする。
def comparing_blds(target_column,df):
    ls_pref=list(df.pref_name.unique())
    ratio_target_column=str("ratio_"+target_column)
    rank_target_column=str("rank_"+target_column)
    benchmark_target=df.iloc[1,:][target_column]
    
    df_agg=pd.DataFrame()
    for i in range(len(ls_pref)):
        # 新規物件のtarget_columnの値と相対評価用のテーブルのtarget_columnの差の絶対値を求め、昇順にソートし、均等にスコアを割り当てる（例：１０物件あったら0.1刻みで0.1から１まで）
        pref_name=ls_pref[i]
        df_atbb_candidate_pref=df.query('pref_name==@pref_name')
        df_atbb_candidate_pref.loc[:, ratio_target_column] = abs(df_atbb_candidate_pref[target_column] - benchmark_target)

        df_atbb_candidate_for_scorig_sorted=df_atbb_candidate_pref.sort_values(ratio_target_column).reset_index()
        df_atbb_candidate_for_scorig_sorted.loc[:,rank_target_column]=df_atbb_candidate_for_scorig_sorted.index
        df_atbb_candidate_for_scorig_sorted.loc[:,rank_target_column]= round(0.7*(1-(df_atbb_candidate_for_scorig_sorted[rank_target_column]*100/len(df_atbb_candidate_pref))/100),2)
        
        # 後処理
        df_atbb_candidate_for_scorig_sorted.drop(columns={"index"},inplace=True)
        df_atbb_candidate_for_scorig_sorted.drop_duplicates(["address","longitude","latitude"],inplace=True)
        df_agg=pd.concat([df_agg,df_atbb_candidate_for_scorig_sorted])
        
    df_agg=df_agg.reset_index().drop(columns={"index"})
    
    return df_agg


# スコア計算
def calc_score(df_atbb_candidate,df_benchmark,distance_from_station,I_amounts,benchmark_store_name):
    # スコアを最大１にする
    func=lambda x: x if x <= 1 else (round((-5/16*x+5/4),3) if x<=4 else 0) 

    df_atbb_candidate.loc[:,"distance_1_ratio"]=round((df_atbb_candidate["distance_from_station"]+1)/distance_from_station,3)
    
    # 欠損値処理
    df_atbb_candidate=df_atbb_candidate.dropna(how='any')
    df_atbb_candidate=df_atbb_candidate.query('　not (geodemo1==0 and geodemo2==0 and geodemo3==0 and geodemo4==0 and  geodemo5==0 \
                                                    and geodemo6==0 and geodemo7==0 and geodemo8==0 and geodemo9==0 and geodemo10==0)')
    df_atbb_candidate=df_atbb_candidate.query('　not (w0_3==0 and w3_6==0 and w6_9==0 and w9_12==0 and w12_15==0 and w15_18==0 and w18_21==0 and w21_24==0 and \
                                                    h0_3==0 and h3_6==0 and h6_9==0 and h9_12==0 and h12_15==0 and h15_18==0 and h18_21==0 and h21_24==0 and asahiru_totalpop==0)')
    
    # 各店舗に対して類似度計算
    def calc_similarity():
        n=2
        
        idx_geodemo1=df_atbb_candidate.columns.get_loc('geodemo1')
        idx_asahiru=df_atbb_candidate.columns.get_loc('w0_3')
        idx_pop_ratio=df_atbb_candidate.columns.get_loc('asahiru_totalpop_ratio')
        
        ls_geodemo=[]
        ls_asahiru=[]
        ls_bld_name=list(df_atbb_candidate["bld_name"])
        for j in tqdm(range(len(df_atbb_candidate))):#
            score_geodemo=math.floor(cos_sim(df_benchmark[4:(4+11)],df_atbb_candidate.iloc[j,idx_geodemo1-1:(idx_geodemo1+10)])*10**n)/(10**n)
            score_asahiru_tmp=math.floor(cos_sim(df_benchmark[16:(16+16)],df_atbb_candidate.iloc[j,idx_asahiru:(idx_asahiru+16)])*10**n)/(10**n)
            score_asahiru_tmp_2=df_atbb_candidate.iloc[j,idx_pop_ratio]
            score_asahiru=math.floor((score_asahiru_tmp*0.3+score_asahiru_tmp_2*0.7)*10**n)/(10**n)

            ls_geodemo.append(score_geodemo)
            ls_asahiru.append(score_asahiru)
        
        return ls_geodemo,ls_asahiru
    
    
    # DataFrame作成のためのリスト作成
    ls_geodemo,ls_asahiru=calc_similarity()
    ls_distance_from_station_1=list(df_atbb_candidate["distance_1_ratio"])
    ls_bld_name=list(df_atbb_candidate.bld_name)
    ls_longitude=list(df_atbb_candidate["longitude"])
    ls_latitude=list(df_atbb_candidate["latitude"])
    ls_address=list(df_atbb_candidate["address"])
    
    df_atbb_candidate.loc[:,"retail_amount_ratio"]=round((df_atbb_candidate["I_amounts"]+1)/I_amounts,3)
    ls_retail_amount_ratio=df_atbb_candidate["retail_amount_ratio"]
    
    # 類似度計算結果のデータフレーム作成
    df_similarity=pd.DataFrame({
        "bld_name":ls_bld_name,
        "address":ls_address,
        "score_access_no1":ls_distance_from_station_1,
        "score_geodemo":ls_geodemo,
        "score_asahiru":ls_asahiru,
        "retail_amount_ratio":ls_retail_amount_ratio,
        "longitude":ls_longitude,
        "latitude":ls_latitude
    })
    df_similarity=df_similarity.reset_index().drop(columns={"index"})
    df_similarity.loc[:,"score_access"]=df_similarity["score_access_no1"].map(func)
    df_similarity.loc[:,"score_retail_amount"]=df_similarity["retail_amount_ratio"].map(func)
    
    # 結果を結合
    df_similarity_for_concat=df_similarity.drop(columns={"score_access_no1","retail_amount_ratio"})
    df_result_tmp=pd.merge(df_atbb_candidate,df_similarity_for_concat,how="left",on=["bld_name","address","longitude"])
    # df_result_tmp_2=df_result_tmp.drop(columns={"latitude_y"}).rename(columns={"longitude_x":"longitude","latitude_x":"latitude"})
    df_result_tmp.drop(columns={"latitude_y"},inplace=True)
    df_result_tmp.rename(columns={"longitude_x":"longitude","latitude_x":"latitude"},inplace=True)
    
    # 総合点算出
    df_result_tmp.loc[:,"total_score"]=round((df_result_tmp["score_geodemo"]+df_result_tmp["score_access"]+df_result_tmp["score_asahiru"]+df_result_tmp["score_retail_amount"])/4,3)

    # レポート用にカラムを分かりやすくする
    df_result_tmp_2=df_result_tmp.reindex(columns=["pref_name","bld_name","address","total_score","score_geodemo",
                                    "score_asahiru","score_access","score_retail_amount","geodemo1", 'geodemo2', 'geodemo3', 'geodemo4',
                                       'geodemo5', 'geodemo6', 'geodemo7', 'geodemo8', 'geodemo9', 'geodemo10', 'w0_3', 'w3_6', 'w6_9', 'w9_12',
                                       'w12_15', 'w15_18', 'w18_21', 'w21_24', 'h0_3', 'h3_6', 'h6_9', 'h9_12',
                                       'h12_15', 'h15_18', 'h18_21', 'h21_24', 'asahiru_totalpop','distance_from_station',
                                     'distance_1_ratio','retail_amount_ratio',"longitude","latitude"
                                    ])

    
    df_result_one_benchmark=df_result_tmp_2.rename(columns={'pref_name':"都道府県", 'bld_name':"物件名",'address':"住所", 'total_score':"類似度総合点",
           'score_geodemo':"周辺居住者類似度", 'score_asahiru':"滞在人口類似度", 'score_access':"最寄駅からの距離類似度",
           'score_retail_amount':"商業集積類似度", 'geodemo1':"geodemo1比率", 'geodemo2':"geodemo2比率",
           'geodemo3':"geodemo3比率", 'geodemo4':"geodemo4比率", 'geodemo5':"geodemo5比率", 'geodemo6':"geodemo6比率", 'geodemo7':"geodemo7比率", 'geodemo8':"geodemo8比率",
           'geodemo9':"geodemo9比率", 'geodemo10':"geodemo10比率",'w0_3':"平日０−３時の滞在人口", 'w3_6':"平日３−６時の滞在人口",
           'w6_9':"平日６−９時の滞在人口", 'w9_12':"平日９−１２時の滞在人口", 'w12_15':"平日１２−１５時の滞在人口", 'w15_18':"平日１５−１８時の滞在人口", 'w18_21':"平日１８−２１時の滞在人口", 'w21_24':"平日２１−２４時の滞在人口", \
            'h0_3':"休日０−３時の滞在人口", 'h3_6':"休日３−６時の滞在人口",
           'h6_9':"休日６−９時の滞在人口", 'h9_12':"休日９−１２時の滞在人口", 'h12_15':"休日１２−１５時の滞在人口", 'h15_18':"休日１５−１８時の滞在人口", 'h18_21':"休日１８−２１時の滞在人口", 'h21_24':"休日２１−２４時の滞在人口",
           'asahiru_totalpop':"滞在人口合計", 'distance_from_station':"最寄駅からの距離","distance_1_ratio":"最寄駅からの距離の比率","retail_amount_ratio":"小売業の売上の比率"})
    
    df_result_one_benchmark.sort_values("類似度総合点",ascending=False).to_csv(f'result/df_result_{benchmark_store_name}.csv',index=False,encoding="utf-8")
    
    return df_result_one_benchmark


# 計算結果をまとめる
def agg_calc_score(df_atbb_candidate,df_benchmark):
    df_benchmarks = {}
    for n in range(1,len(df_benchmark)):
        df_benchmarks[f"df_benchmark_{n}"] = df_benchmark.iloc[n, :]
    
    df_result_agg=pd.DataFrame()
    
    # ベンチマーク店舗ごとに類似度を計算
    for n in range(1,len(df_benchmark)):
        benchmark_i_amounts=df_benchmarks[f"df_benchmark_{n}"]["I_amounts"]
        benchmark_pop=df_benchmarks[f"df_benchmark_{n}"][16:32].mean()
        benchmark_sta=df_benchmarks[f"df_benchmark_{n}"]["distance_from_station"]
    
        df_atbb_candidate=calc_ratio_population(df_atbb_candidate,benchmark_pop)
        df_calc_one_benchmark=calc_score(df_atbb_candidate,df_benchmarks[f"df_benchmark_{n}"],benchmark_sta,benchmark_i_amounts,"benchmark_2")
    
        df_result_agg=pd.concat([df_result_agg, df_calc_one_benchmark])
    
    # ベンチマーク店舗ごとに計算した類似度をまとめる
    df_result_agg_mean=df_result_agg.groupby(["都道府県","物件名","住所","longitude","latitude"]) \
                                    .mean()[["類似度総合点","周辺居住者類似度","滞在人口類似度","最寄駅からの距離類似度","商業集積類似度"]] \
                                    .sort_values('類似度総合点',ascending=False)
    df_result_agg_mean=df_result_agg_mean.reset_index()[["都道府県","物件名","住所","類似度総合点","周辺居住者類似度","滞在人口類似度","最寄駅からの距離類似度","商業集積類似度","longitude","latitude"]]
        
    return df_result_agg_mean


# スコア調整（0.3,0.7の比率は経験則）
def scoring(df_result_agg_mean,df_master):
    # 商業集積の類似度　
    df_result_agg_mean.loc[:,"商業集積類似度"]=df_result_agg_mean["商業集積類似度"].apply(lambda x: x*0.3)
    df_result_agg_mean_merge=pd.merge(df_result_agg_mean,df_master[["住所","longitude","latitude","rank_I_amounts"]],on=["住所","longitude","latitude"],how="left")
    df_result_agg_mean_merge.loc[:,"商業集積類似度"]=df_result_agg_mean_merge["商業集積類似度"]+df_result_agg_mean_merge["rank_I_amounts"]
    
    # 最寄駅からの距離類似度
    df_result_agg_mean_merge.loc[:,"最寄駅からの距離類似度"]=df_result_agg_mean["最寄駅からの距離類似度"].apply(lambda x: x*0.3)
    df_result_agg_mean_merge=pd.merge(df_result_agg_mean_merge,df_master[["住所","longitude","latitude","rank_distance_from_station"]],on=["住所","longitude","latitude"],how="left")
    df_result_agg_mean_merge.loc[:,"最寄駅からの距離類似度"]=df_result_agg_mean_merge["最寄駅からの距離類似度"]+df_result_agg_mean_merge["rank_distance_from_station"]
    
    # 総合店算出
    df_result_agg_mean_merge.drop(columns={"rank_distance_from_station"},inplace=True)
    df_result_agg_mean_merge.loc[:,"類似度総合点"]=round((df_result_agg_mean_merge["周辺居住者類似度"]+df_result_agg_mean_merge["滞在人口類似度"] \
                                                        +df_result_agg_mean_merge["最寄駅からの距離類似度"]+df_result_agg_mean_merge["商業集積類似度"])/4,3)
    
    return df_result_agg_mean_merge


# 最後に提出用として整理
def separte_result_by_pref(df,brand_name,today_date):
    ls_pref=list(df.都道府県.unique())
    df=df.sort_values('類似度総合点',ascending=False)
    for i in range(len(ls_pref)):
        pref_name=ls_pref[i]
        df_result=df.query('都道府県==@pref_name')
        df_result.loc[:, "address"] = df_result["住所"] + "_" + df_result["物件名"]

        df_result.to_csv(f'result/{brand_name}/{brand_name}_results_{today_date}/df_result_{pref_name}_{today_date}.csv',index=False,encoding="utf-8")
        
    return df_result


def benchmark_geo_asa_income_sta(brand_name):
    query=f'''
    select *
    from geom-prj-property-recommend.{brand_name}.{brand_name}_geo_asa_income_sta
    '''
    
    df=client.query(query).to_dataframe()
    
    return df


## Load data and Calc score

In [14]:
ls_brand_name=["","LifeCoach","fukushin","bestbuy","tamura"]
today_date=datetime.datetime.now().strftime('%Y%m%d')

###########
#days=-7
#target_date=(datetime.datetime.now() + datetime.timedelta(days=days)).strftime('%Y%m%d') #データ取得日と計算実施日がズレることがあるので
target_date = '20230508' # BigQueryの対象のデータと同じ日付にする。
###########

for i in tqdm(range(1,len(ls_brand_name))):
    
    brand_name=ls_brand_name[i]
    print(brand_name)
    
    # フォルダ作成
    new_dir_path = f'result/{brand_name}/{brand_name}_results_{today_date}'
    try:
        os.mkdir(new_dir_path)
    except:
        pass
    
    # テーブルを読み込み
    df_atbb_candidate=agg_geodemo_asahiru_income_station(brand_name,f"{brand_name}_atbb_candidate_geocoding_{target_date}") # 新規に掲載された物件
    df_atbb_scoring=agg_geodemo_asahiru_income_station(brand_name,f"{brand_name}_atbb_candidate_geocoding_for_scoring") # 1週間以上前の物件。相対評価用
    df_benchmark=benchmark_geo_asa_income_sta(brand_name) # ベンチマーク物件
    
    # 相対評価用のスコアを計算
    df_atbb_scoring=comparing_blds("I_amounts",df_atbb_scoring)
    df_master=comparing_blds("distance_from_station",df_atbb_scoring)
    df_master.rename(columns={"bld_name":"物件名","address":"住所"},inplace=True)
    
    # 絶対評価用のスコアを計算
    df_result_agg_mean=agg_calc_score(df_atbb_candidate,df_benchmark)
    
    # 二つの評価をまとめる
    df_result_agg_mean_merge=scoring(df_result_agg_mean,df_master)
    df_result=separte_result_by_pref(df_result_agg_mean_merge,brand_name,today_date)
    

  0%|          | 0/4 [00:00<?, ?it/s]

LifeCoach



100%|██████████| 3/3 [00:00<00:00, 1357.38it/s]

 25%|██▌       | 1/4 [00:07<00:21,  7.16s/it]

fukushin



100%|██████████| 5/5 [00:00<00:00, 1375.90it/s]

100%|██████████| 5/5 [00:00<00:00, 1473.75it/s]

100%|██████████| 5/5 [00:00<00:00, 1409.76it/s]

 50%|█████     | 2/4 [00:13<00:13,  6.95s/it]

bestbuy


NotFound: 404 Not found: Table geom-prj-property-recommend:bestbuy.bestbuy_atbb_candidate_geocoding_20230508 was not found in location asia-northeast1

Location: asia-northeast1
Job ID: a783130f-91b2-47d0-8efc-4d75fe06ebd1


## 人口統計（レコメンド関係ない）

In [15]:
def func():
    query =f"""
        select c.* -- ,m.bq_centroidgeom
        from `geomarketing-analysis-359502.murakami_asia.cohort_allgen` c
        -- left join `geomarketing-common.public_asia.chomoku2015` m
        -- on c.chomokucode=cast(m.chomokucode as int64)
        -- where st_distance(bq_centroidgeom,st_geogpoint(135.7907972915824,34.725130692990454)) <= 10000
        """

    df=client.query(query).to_dataframe()
    
    return df

In [ ]:
df=func()

In [ ]:
df["sum_senior"]=df["p65_69t"]+df["p70_74t"]+df["p75_79t"]+df["p80_84t"]+df["p85_89t"]+df["p90_94t"]+df["p95_99t"]+df["p100t"]
df["sum_junior"]=df["p00_04t"]+df["p05_09t"]+df["p10_14t"]+df["p15_19t"]+df["p20_24t"]+df["p25_29t"]+df["p30_34t"]+df["p35_39t"]+df["p40_44t"]+df["p45_49t"]+df["p50_54t"]+df["p55_59t"]+df["p60_64t"]
df["total_pop"]=df["sum_senior"]+df["sum_junior"]


In [ ]:
df=df[["chomokucode","total_pop"]]

In [ ]:
df.sort_values("total_pop")

In [ ]:
project_name="geomarketing-analysis-359502" # 保存先のプロジェクト名
dataset="murakami_asia" # 保存先のデータセット名
table_name=""

def upload_bq(dataset,table_name,df):
    table_id = f'{project_name}.{dataset}.{table_name}'
    df.to_gbq(table_id)